konlpy 설치
- 환경변수 편집 jvm.dll 경로 확인 : C:\Program Files\Java\jdk1.8.0_261\jre\bin\server

https://data-scientist-brian-kim.tistory.com/79


In [1]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import konlpy

In [2]:
import pandas as pd
train_df = pd.read_csv('./dataset/ratings_train.txt',sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
train_df.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

## 데이터 전처리

In [4]:
import re

train_df = train_df.fillna(' ')

In [5]:
train_df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [6]:
train_df['document']=train_df['document'].apply(lambda x:re.sub(r'\d+',' ',x))

In [7]:
test_df=pd.read_csv('./dataset/ratings_test.txt',sep='\t')
test_df=test_df.fillna(' ')
test_df['document']=test_df['document'].apply(lambda x:re.sub(r'\d+',' ',x))

In [8]:
test_df=test_df.drop('id',axis=1)

In [9]:
test_df.head()

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,D만 아니었어도 별 다섯 개 줬을텐데.. 왜 D로 나와서 제 심기를 불편하게 하죠??,0


In [10]:
from konlpy.tag import Okt
okt= Okt()
def okt_tokenizer(text):
    #입력 인자로 들어온 text를 형태소 단어로 토큰화하여 list객체 반환
    tokens_ko=okt.morphs(text)
    return tokens_ko

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3,max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train=tfidf_vect.transform(train_df['document'])

D:\anaconda3\envs\cakd5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
ipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10, max_iter = 500))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))